In [2]:
import polars as pl
import constants
import psycopg
from riotwatcher import LolWatcher, ApiError, RiotWatcher

# sys.stdout.reconfigure(encoding='utf-8')

API_KEY = constants.API_KEY_SERVICE
lol_watcher = LolWatcher(API_KEY)
riot_watcher = RiotWatcher(API_KEY)
lol_region = 'na1'

db_pass = constants.db_password
db_ip = constants.db_ip

db_connection = f'dbname = yrden user=postgres password={db_pass} host={db_ip}'

conn = psycopg.connect(db_connection)

In [7]:
stats_query = '''SELECT * FROM "yrden".lol_game_data
WHERE game_id in 
(select game_id
from "yrden".lol_game_data
where 1=1
and queue_id in (440, 700)
group by game_id
having count(game_id) = 10)
ORDER BY GAME_PATCH DESC, GAME_ID, TEAMID, CASE WHEN LANE = 'TOP' THEN 1 WHEN LANE = 'JUNGLE' THEN 2 WHEN LANE = 'MIDDLE' THEN 3 WHEN LANE = 'BOTTOM' THEN 4 ELSE 5 END
;'''

with conn.cursor() as cur:
    cur.execute(stats_query)
    col_names = [desc[0] for desc in cur.description]
    rows = cur.fetchall()
    df = pl.DataFrame(rows, schema=col_names, orient='row')

df.shape

(230, 59)

In [86]:
yrden_df = df.filter(pl.col('riot_puuid').is_in(constants.yrden_lol_team_puuids))

In [87]:
champ_df = yrden_df.select(pl.col('game_id'), pl.col('win'), pl.col('champion_id'), pl.col('riot_id'), pl.col('riot_puuid'))

In [88]:
champ_df

game_id,win,champion_id,riot_id,riot_puuid
str,bool,i64,str,str
"""NA1_5206375709""",true,122,"""Hypocritus""","""Xhi01hd4YPU92OGkFKSsqEnTtVwTD_…"
"""NA1_5206375709""",true,72,"""Triggerman""","""0CpS1bKgKnH7IKuMVnpeOpyLFIj3J7…"
"""NA1_5206375709""",true,103,"""Blue""","""gw2MWsJzlQYIRr40fYEyB2RdJBixmb…"
"""NA1_5206375709""",true,202,"""YDN Rock Coaches""","""oFdCaHSdLE-umAd_AUTwskzbTOEPp-…"
"""NA1_5206375709""",true,143,"""wyzrdsnvrdie""","""nQd7Ylk472T9NtwuS-hhuLDcxoUg71…"
…,…,…,…,…
"""NA1_5177097514""",true,122,"""Hypocritus""","""Xhi01hd4YPU92OGkFKSsqEnTtVwTD_…"
"""NA1_5177097514""",true,113,"""Triggerman""","""0CpS1bKgKnH7IKuMVnpeOpyLFIj3J7…"
"""NA1_5177097514""",true,163,"""Blue""","""gw2MWsJzlQYIRr40fYEyB2RdJBixmb…"


In [153]:
pivot_df = champ_df.pivot('riot_id', index='game_id')
pivot_df.select_seq(pl.col('game_id'), pl.col('win_Blue').alias('win'), pl.col('champion_id_Hypocritus')|
                pl.col('champion_id_Triggerman')| pl.col('champion_id_Blue')| pl.col('champion_id_YDN Rock Coaches')|
                pl.col('champion_id_wyzrdsnvrdie'))

game_id,win,champion_id_Hypocritus
str,bool,i64
"""NA1_5206375709""",true,255
"""NA1_5206412184""",false,1023
"""NA1_5207643890""",false,447
"""NA1_5207671213""",false,1023
"""NA1_5205572005""",false,127
…,…,…
"""NA1_5168606154""",true,1023
"""NA1_5170644901""",true,255
"""NA1_5170720153""",true,1023


In [186]:
concat_df = champ_df.with_columns(pl.col('champion_id')
                                .str.concat(delimiter=',')
                                .over('game_id')
                                .alias('concatenated_champ_ids'))

converted_df = concat_df.with_columns(
    pl.col('concatenated_champ_ids')
    .str.split_exact(',',5)
    .cast(pl.Struct([pl.Field(f'field_{i}', pl.Int64) for i in range(5)]))
    .alias('champ_id_struct')
)

final_df = converted_df.select(['game_id', 'win', 'champ_id_struct']).unique()
final_df

game_id,win,champ_id_struct
str,bool,struct[5]
"""NA1_5197758341""",true,"{516,104,103,202,89}"
"""NA1_5177097514""",true,"{122,113,163,222,111}"
"""NA1_5177050727""",true,"{516,62,103,15,143}"
"""NA1_5187428841""",true,"{122,113,163,51,143}"
"""NA1_5206375709""",true,"{122,72,103,202,143}"
…,…,…
"""NA1_5198519859""",false,"{516,59,518,51,99}"
"""NA1_5197728845""",false,"{122,64,61,22,201}"
"""NA1_5183176803""",true,"{516,104,518,202,89}"


In [204]:
test = final_df.with_columns(
    pl.when(pl.col('win') == True)
    .then(1)
    .otherwise(0)
    .alias('win_flag')
)

result = test.group_by('champ_id_struct', maintain_order=True).agg([pl.col('win_flag').eq(1).sum().alias('wins'),
                                      pl.count('game_id').alias('total_matches')]).with_columns((pl.col('wins')/pl.col('total_matches')).alias('winrate'))

result.write_excel('result.xlsx')

In [199]:
test.write_excel('polars_data.xlsx')